In [1]:
import numpy as np
import torch
from script_numba.reformulate_weights import gates
from script_numba.GBM import growth_bounds_matrix
import warnings
warnings.filterwarnings("ignore")


## Inputs

In [2]:
ka = 0.6
kb = 0.1

In [3]:
VV = np.load('./inputs/x_interval.npy')
HH = np.load(f'./model/imdb/glove/{ka}_{kb}/h_interval.npy')
CC = np.load(f'./model/imdb/glove/{ka}_{kb}/c_interval.npy')

model = torch.load(f'./model/imdb/glove/{ka}_{kb}/model.pth')

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig_x = sigmoid(x)
    return sig_x * (1 - sig_x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

In [9]:
def interval_multiplication(array1, array2):
    # Unpack the lower and upper bounds
    x1, x2 = array1[..., 0], array1[..., 1]
    y1, y2 = array2[..., 0], array2[..., 1]
    
    # Compute all possible products
    values1 = x1 * y1
    values2 = x1 * y2
    values3 = x2 * y1
    values4 = x2 * y2
    
    # Calculate min and max for each interval using NumPy's efficient min and max functions
    interval_min = np.minimum.reduce([values1, values2, values3, values4], axis=0)
    interval_max = np.maximum.reduce([values1, values2, values3, values4], axis=0)
    
    return np.stack((interval_min, interval_max), axis=-1)

In [6]:
# Define the CUDA kernel
@cuda.jit
def float_multiplication_kernel(matrix, interval, result):
    i, j = cuda.grid(2)
    
    if i < result.shape[0] and j < result.shape[1]:
        # p = matrix[i, j] * interval[i]
        # if matrix[i, j] >= 0:
        #     result[i, j] = p
        # else:
        #     result[i, j] = matrix[i][j] * interval[j][V.shape[1]-1-k]
   
        for k in range(interval.shape[1]):
            if matrix[i][j] >= 0:
                result[i, j][k] = matrix[i][j] * interval[i][k]
            else:
                result[i, j][k] = matrix[i][j] * interval[i][interval.shape[1]-1-k]

In [7]:
def float_multiplication_optimized(matrix, interval):
    # Move data to device
    matrix_dev = cuda.to_device(matrix)
    interval_dev = cuda.to_device(interval) 
    result_dev = cuda.device_array((matrix.shape[0], matrix.shape[1],2), dtype=np.float32) 

    # Define grid and block sizes
    threads_per_block = (16, 16)  # Define appropriate block size
    blocks_per_grid_x = (matrix.shape[0] + threads_per_block[0] - 1) // threads_per_block[0]
    blocks_per_grid_y = (matrix.shape[1] + threads_per_block[1] - 1) // threads_per_block[1]
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    # Launch the kernel
    float_multiplication_kernel[blocks_per_grid, threads_per_block](matrix_dev, interval_dev, result_dev)

    # Copy the result back to the host
    result = result_dev.copy_to_host()
    return result


In [8]:
def C2_t(V,H,C):
    
    f = sigmoid(np.dot(forget_gate[0][0],V) + np.dot(forget_gate[0][1],H) + forget_gate[1])
    I = sigmoid(np.dot(input_gate[0][0],V) + np.dot(input_gate[0][1],H) + input_gate[1])
    g = np.tanh(np.dot(cell_gate[0][0],V) + np.dot(cell_gate[0][1],H) + cell_gate[1])
    
    return np.multiply(f,C) + np.multiply(I,g)

In [9]:
def proposition_2(fct,interval):
    result=[]
    for i in range (interval.shape[0]):
        a = interval[i][0]
        b = interval[i][1]
        min_fct = np.min([fct(a), fct(b)])
        if a<=0 and b >=0:
            max_fct = fct(0)
        else:
            max_fct = np.max([fct(a), fct(b)])
        result.append([min_fct,max_fct])
    return np.array(result)

In [10]:
@cuda.jit
def proposition_4_kernel(omega, U, V, H, bias, T):
    i = cuda.grid(1)
    max_omega_cols = 300  # Use the maximum possible size for omega columns
    max_U_cols = 64       # Use the maximum possible size for U columns 
    if i < omega.shape[0]:
        V_gate_i = cuda.local.array((max_omega_cols, 2), dtype=float32)
        H_gate_i = cuda.local.array((max_U_cols, 2), dtype=float32) 
        
        # Compute V_gate_i
        for j in range(omega.shape[1]):
            for k in range(V.shape[1]):
                if omega[i][j] >= 0:
                    V_gate_i[j][k] = omega[i][j] * V[j][k]
                else:
                    V_gate_i[j][k] = omega[i][j] * V[j][V.shape[1]-1-k]
        
        # Compute H_gate_i
        for l in range(U.shape[1]):
            for k in range(H.shape[1]):
                if U[i][l] >= 0:
                    H_gate_i[l][k] = U[i][l] * H[l][k]
                else:
                    H_gate_i[l][k] = U[i][l] * H[l][H.shape[1]-1-k]
        
        # Sum V_gate_i and H_gate_i
        sum_V = cuda.local.array(2, dtype=float32)
        sum_H = cuda.local.array(2, dtype=float32)
        for k in range(2):
            sum_V[k] = 0.0
            sum_H[k] = 0.0
        
        for j in range(omega.shape[1]):
            for k in range(V.shape[1]):
                sum_V[k] += V_gate_i[j][k]
                
        for l in range(U.shape[1]):
            for k in range(H.shape[1]):
                sum_H[k] += H_gate_i[l][k]
        
        for k in range(T.shape[1]):
            T[i][k] = sum_V[k] + sum_H[k] + bias[i]


def proposition_4_optimized(output_gate_0_0, output_gate_0_1, VV, HH, output_gate_1):
    # Move data to device
    omega_dev = cuda.to_device(output_gate_0_0)
    U_dev = cuda.to_device(output_gate_0_1)
    V_dev = cuda.to_device(VV)
    H_dev = cuda.to_device(HH)
    bias_dev = cuda.to_device(output_gate_1)
    T_dev = cuda.device_array((omega_dev.shape[0], 2), dtype=np.float32) 
    
    # Define grid and block sizes
    threads_per_block = 32
    blocks_per_grid = (64 + threads_per_block - 1) // threads_per_block

    # Launch the kernel
    proposition_4_kernel[blocks_per_grid, threads_per_block](omega_dev, U_dev, V_dev, H_dev, bias_dev, T_dev)

    # Copy the result back to the host
    T = T_dev.copy_to_host()
    return T

In [11]:
def proposition_5(ct,V,H,C,A,B,D):
    under_V = V[:,0]
    bar_V = V[:,1]
    
    under_H = H[:,0]
    bar_H = H[:,1]
    
    under_C = C[:,0]
    bar_C = C[:,1]
    
    mid_V = (under_V + bar_V)/2
    mid_H = (under_H + bar_H)/2
    mid_C = (under_C + bar_C)/2
    
    diff_V = bar_V - under_V
    diff_H = bar_H - under_H
    diff_C = bar_C - under_C

    c = []
    Ct = ct(mid_V,mid_H,mid_C)
 
    for i in range(C.shape[0]):
        min_ci = Ct[i] - np.dot(A[i], diff_V) - np.dot(B[i], diff_H) - np.dot(D[i], diff_C)
        max_ci = Ct[i] + np.dot(A[i], diff_V) + np.dot(B[i], diff_H) + np.dot(D[i], diff_C)
        c.append([min_ci, max_ci])
    return np.array(c)

In [12]:
def GBM(BM):
    return np.max(np.abs(BM), axis=-1)

In [ ]:
T_o = proposition_4_optimized(output_gate[0][0], output_gate[0][1], VV, HH, output_gate[1])
T_f = proposition_4_optimized(forget_gate[0][0], forget_gate[0][1], VV, HH, forget_gate[1])
T_I = proposition_4_optimized(input_gate[0][0], input_gate[0][1], VV, HH, input_gate[1])
T_g = proposition_4_optimized(cell_gate[0][0], cell_gate[0][1], VV, HH, cell_gate[1])

sigma_Tf = sigmoid(T_f)
sigma_TI = sigmoid(T_I)
sigma_To = sigmoid(T_o)
tanh_Tg = np.tanh(T_g)

# ct = C_t(T_f, T_I, T_g, CC)

### Bounding $C_t$

#### $$\frac{\partial c_t}{\partial v_{wt}} \quad , \quad \frac{\partial c_t}{\partial h_{t-1}} \quad and \quad \frac{\partial c_t}{\partial c_{t-1}}$$ 

In [14]:
def jaccobian_C(variable):
    if variable == 'v':
        var = 0
    elif variable == 'h':
        var = 1
    elif variable == 'c':
        T1 = sigma_Tf.copy()
        T2 = T1[np.newaxis,:,:]
        return np.repeat(T2, 64, axis=0)
    else:
        return -1
    
    T11 = interval_multiplication(proposition_2(sigmoid_derivative, T_f), CC)
    T1 = float_multiplication_optimized(forget_gate[0][var], T11)
    # T1 = float_multiplication(forget_gate[0][var], T11)

    T21 = interval_multiplication(proposition_2(sigmoid_derivative, T_I), tanh_Tg)
    T2 = float_multiplication_optimized(input_gate[0][var], T21)
    # T2 = float_multiplication(input_gate[0][var], T21)

    T31 = interval_multiplication(proposition_2(tanh_derivative, T_g), sigma_TI)
    T3 = float_multiplication_optimized(cell_gate[0][var], T31)
    # T3 = float_multiplication(cell_gate[0][var], T31)

    return T1 + T2 + T3

In [15]:
C_v = jaccobian_C('v')
C_h = jaccobian_C('h')
C_c = jaccobian_C('c')

In [16]:
Ac = GBM(C_v)
Bc = GBM(C_h)
Dc = GBM(C_c)

-------------------------

### Bounding $H_t$

#### $$\frac{\partial h_t}{\partial v_{wt}} \quad , \quad \frac{\partial h_t}{\partial h_{t-1}}\quad and \quad \frac{\partial h_t}{\partial c_{t-1}}$$

In [17]:
def jaccobian_H(variable):
    if variable == 'v':
        var = 0
        var2 = C_v
    elif variable == 'h':
        var = 1
        var2 = C_h
    elif variable == 'c':
        T1 = interval_multiplication(interval_multiplication(sigma_To, sigma_Tf), proposition_2(tanh_derivative,proposition_5(C2_t,VV,HH,CC,Ac,Bc,Dc)))
        T2 = T1[np.newaxis,:,:]
        return np.repeat(T2, 64, axis=0)
    else:
        return -1
 
    c = proposition_5(C2_t,VV,HH,CC,Ac,Bc,Dc)
    T11 = interval_multiplication(proposition_2(sigmoid_derivative, T_o), np.tanh(c))
    T1 = float_multiplication_optimized(output_gate[0][var], T11)

    T21 = interval_multiplication(proposition_2(tanh_derivative, c), sigma_To)
    T22 = T21.copy()
    T23 = T22[:,np.newaxis,:]
    T2 = interval_multiplication(var2, np.repeat(T23, output_gate[0][var][1].shape, axis=1))
    
    return T1 + T2

In [18]:
H_v = jaccobian_H('v')
H_h = jaccobian_H('h')
H_c = jaccobian_H('c')

In [19]:
A = GBM(H_v)
B = GBM(H_h)
D = GBM(H_c)

## Test script

In [4]:
weights = [model[param].cpu().detach().numpy() for param in ['lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0']]
weights_reverse = [model[param].cpu().detach().numpy() for param in ['lstm.weight_ih_l0_reverse', 'lstm.weight_hh_l0_reverse', 'lstm.bias_ih_l0_reverse', 'lstm.bias_hh_l0_reverse']]

In [5]:
A,B,D = growth_bounds_matrix(weights, VV,HH[0],CC[0],64)
A_r,B_r,D_r = growth_bounds_matrix(weights_reverse, VV,HH[1],CC[1],64)

In [6]:
np.sum(np.sum(A, axis=1))

11.814072960924685

In [7]:
np.sum(np.sum(A_r, axis=1))

1095.250707646067

In [8]:
np.sum(np.sum(B, axis=1))

0.33880039991572386

In [9]:
np.sum(np.sum(B_r, axis=1))

406.81301927801786

In [10]:
np.sum(D[0])

18.16561210155487

In [11]:
np.sum(D_r[0])

62.466596215963364

In [24]:
h = np.full((64,2),[np.min(HH[0]),np.max(HH[0])])
c = np.full((64,2),[np.min(CC[0]),np.max(CC[0])])
x = np.full((300,2),[np.min(VV), np.max(VV)] )

h1 = np.full((64,2),[np.min(HH[1]),np.max(HH[1])])
c1 = np.full((64,2),[np.min(CC[1]),np.max(CC[1])])

In [25]:
A_1,B_1,D_1 = growth_bounds_matrix(weights, x,h,c,64)
A_1_r,B_1_r,D_1_r = growth_bounds_matrix(weights_reverse, x,h1,c1,64)

In [26]:
np.sum(np.sum(A_1, axis=1))

6.776769574047858

In [27]:
np.sum(np.sum(A_1_r, axis=1))

1176.2027931560228

In [28]:
np.sum(np.sum(B_1, axis=1))

0.08150408580265399

In [29]:
np.sum(np.sum(B_1_r, axis=1))

458.3771369474824

In [31]:
np.sum(D_1[0])

18.38121145963669

In [32]:
np.sum(D_1_r[0])

61.91715118288994